<img src="images/inmas.png" width=130x align='right' />

# Exercises 23 - OpenAI Application Programming Interface
    
This notebook requires that you concurrently use a terminal. Use virtual desktops if possible (e.g., Windows Key+Ctl+arrows).

### Prerequisite
Notebook 23

----

In [ ]:
import wave, struct, os, time
from openai import OpenAI                        # The API to OpenAI
from pvrecorder import PvRecorder                # To record our voice
from playsound import playsound                  # To play the generated speech
from IPython.display import Image, display       # To show generated images

# To silence deprecation warnings
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

### 1.
Experiment with different voices and cues for the personality of the system and for the image generation in the last version of the ChatBot class. You must have noticed that chatgpt and DALL-E have an *unconscious* bias toward male professors: while the gender was never specified, it always returns male images.
- Change the voice to a female or neutral voice (only *onyx* and *echo* are male voices)
- Change the cues to chatgpt (`system`) so that both the personality and the generated image represent a female professor

Voice samples are [here](https://platform.openai.com/docs/guides/text-to-speech)

### 2.
Use a call to 'gpt-4o' to generate code in python to play Hangman. Copy the response message to a cell and run the program to see if it behaves as it should. You will need an API key and some credits on OpenAI to accomplish this task.

### 3.*
This exercise further develops the class ChatBot that was presented in the Notebook. We reproduce it here for your convenience:

```python
class ChatBot:
    '''A class to interact with OpenAI API and keep track of context.'''
    def __init__(self, client, model):
        self.client = client
        self.model = model
        self.context = [{'role': 'system', 'content': 'You are math professor with a dark sense of humor'}]

    def chat(self, question):
        self.context.append({'role': 'user', 'content': question})
        response = self.client.chat.completions.create(model=self.model, messages=self.context)
        response_content = response.choices[0].message.content
        self.context.append({'role': 'assistant', 'content': response_content})
        self.print_chat()
        self.speak(response_content, len(self.context)/2)

    def speak(self, message, index=0):
        speech_file = './_speech_%03d.mp3'%index
        response = client.audio.speech.create(model='tts-1-hd', voice='echo', input=message)
        if os.path.exists(speech_file):
            os.remove(speech_file)
        response.stream_to_file(speech_file)
        time.sleep(1)
        playsound(speech_file)   # playsound 1.2.2
        '''
        done = False
        while not done:       # Use this loop instead to get around a bug in `playsound` 1.3.0
            try:
                playsound(speech_file)
                done = True
            except:
                continue
        '''
        
    def print_chat(self):
        for message in self.context:
            if message['role'] == 'user':
                print('USER: %s' % message['content'])
            elif message['role'] == 'assistant':
                print('BOT: %s' % message['content'])
```

We would like to speak to our Bot and convert our speech to text using the *Whisper* model in OpenAI. 

The first step is to record our voice using the following code. It records until time is over. 

```python
index = 0
rec_time = 10
audiofile = './_audio_recording%03d.wav'%index
recorder = PvRecorder(device_index=-1, frame_length=512)
audio = []

recorder.start()
print('Audio recording started: you have %d seconds to ask your question...'%rec_time)
t_end = time.time() + rec_time
while time.time() < t_end:
    frame = recorder.read()
    audio.extend(frame)

recorder.stop()
recorder.delete()

print('Audio recording stopped.')
with wave.open(audiofile, 'w') as f:
    f.setparams((1, 2, 16000, 512, 'NONE', 'not compressed')) # 1=mono, 2=16bit audio, 16KHz, compression, name
    f.writeframes(struct.pack('h'*len(audio), *audio))
```

If you have multiple sound devices on your computer you can list them an choose or use `-1` which picks the default device.

In [ ]:
devices = PvRecorder.get_available_devices()
print('Devices:', devices)

We now use the audiofile which contains our recorded voice and pass it to *Whisper* for transcription:

```python
with open(audiofile, 'r') as f:
    transcript = client.audio.transcriptions.create(model='whisper-1', file=f)

transcript.text
```

Using the content of the last cell for a method called `transcribe(self, audiofile)` which returns the transcribed text to be passed as a message, and the content of the previous one in a method called `record_audio(self, rec_time, index=0)` **which returns the path of the audio file**,  integrate the last two cells in the ChatBot class. One additional method should be added:

```python
def voicechat(self, rec_time):
    recorded_audio = self.record_audio(rec_time, index=len(self.context)/2)   # Use response count for unique filename
    message = self.transcribe(recorded_audio)
    self.chat(message)
```

After defining you `ChatBot` class, you will also need the following to bootstrap the client:

In [ ]:
APIkey = 'YOUR_API_KEY_HERE'

In [ ]:
client = OpenAI(api_key=APIkey)
chatbot = ChatBot(client, 'gpt-4o')

Repeat as many times as you need to keep context. Ask your questions by (re-)running the cell below.

In [ ]:
chatbot.voicechat(3)   # Is 3 seconds enough for your question?